# Stacking des meilleurs modèles

On va faire du stacking des meilleurs modèles avec les hyperparamètres optimisés qu'on a trouvé au notebok précédent. 

On va utiliser un modèle de régression logistique pour faire le stacking.

In [2]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('train.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On divise les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# On normalise les données continues 
from sklearn.preprocessing import StandardScaler

continuous_columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                      'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                      'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
scaler = StandardScaler()

# On applique le scaler uniquement sur les colonnes continues
X_train[continuous_columns] = scaler.fit_transform(X_train[continuous_columns])
X_test[continuous_columns] = scaler.transform(X_test[continuous_columns])

# On ajuste les labels pour qu'ils commencent à 0 pour XGBoost
y_train_adj = y_train - 1
y_test_adj = y_test - 1

In [4]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Définir les modèles de base avec les meilleurs hyperparamètres
random_forest = RandomForestClassifier(
    n_estimators=300,  
    max_depth=None,      
    random_state=42,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
)

xgboost = XGBClassifier(
    colsample_bytree=0.9284956459967804,
    gamma=0.04924511492281908,
    learning_rate=0.24122412239323296,
    max_depth=9,
    n_estimators=300,
    subsample=0.9833984902058166,
    objective='multi:softmax',
    num_class=7,
    random_state=42
)

lightgbm = LGBMClassifier(
    colsample_bytree=0.9488048655607615,
    learning_rate=0.09224159033088929,
    max_depth=15,
    n_estimators=300,
    num_leaves=100,
    subsample=0.5190158468602316,
    objective='multiclass',
    num_class=7,
    random_state=42,
    verbosity=-1
)

# Définir le méta-modèle (un modèle simple comme la régression logistique)
meta_model = LogisticRegression(max_iter=1000)

# Créer le StackingClassifier
stacking_model = StackingClassifier(
    estimators=[
        ('rf', random_forest),
        ('xgb', xgboost),
        ('lgbm', lightgbm)
    ],
    final_estimator=meta_model,
    cv=5  # Utiliser une validation croisée pour le stacking
)

# Entraîner le modèle de stacking
stacking_model.fit(X_train, y_train_adj)

# Prédire sur l'ensemble de test
y_pred_stack = stacking_model.predict(X_test)

# Évaluer les performances du modèle de stacking
accuracy_stack = accuracy_score(y_test_adj, y_pred_stack)
print("Précision du modèle de stacking :", accuracy_stack)
print("Rapport de classification du modèle de stacking :\n", classification_report(y_test_adj, y_pred_stack))

Précision du modèle de stacking : 0.892636684303351
Rapport de classification du modèle de stacking :
               precision    recall  f1-score   support

           0       0.81      0.81      0.81       648
           1       0.80      0.75      0.77       648
           2       0.88      0.91      0.90       648
           3       0.96      0.98      0.97       648
           4       0.93      0.94      0.94       648
           5       0.90      0.92      0.91       648
           6       0.96      0.94      0.95       648

    accuracy                           0.89      4536
   macro avg       0.89      0.89      0.89      4536
weighted avg       0.89      0.89      0.89      4536

